Connor did this... you're welcome 

Download data https://www.kaggle.com/c/nyc-taxi-trip-duration/data

In [1]:
#Add ya code here
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

train = pd.read_csv("train.csv", header=0)
test = pd.read_csv("test.csv", header=0)
#print(train.columns)
#print(test.columns)

In [16]:
# train split to 75/25

split = len(train) * 0.75

train_train = train.loc[:split,:]
train_test = train.loc[split:,:]

print('Taining total set size:',train.shape[0])
print('Training set size:', train_train.shape[0])
print('Test set size:', train_test.shape[0])

Taining total set size: 1458644
Training set size: 1093984
Test set size: 364661


In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test = train_test_split(train, test_size=0.25)

print('Taining total set size:',train.shape[0])
print('Training set size:', X_train.shape[0])
print('Test set size:', X_test.shape[0])

Taining total set size: 1458644
Training set size: 1093983
Test set size: 364661
